## Create an agent in Azure AI Search

This step creates a knowledge agent, which acts as a wrapper for the LLM you deployed to Azure OpenAI. The LLM is used to send queries to an agentic retrieval pipeline.

In [2]:
from azure.search.documents.indexes.models import KnowledgeAgent, KnowledgeAgentAzureOpenAIModel, KnowledgeAgentTargetIndex, KnowledgeAgentRequestLimits, AzureOpenAIVectorizerParameters
from azure.search.documents.indexes import SearchIndexClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

credential = DefaultAzureCredential()
index_name = "dev-mihai-index"
agent_name = "dev-mihai-index-agent"
resource_url="https://cog-dcnxgznrq3wmc.openai.azure.com/"
deployment_name="gpt-4.1"
model_name="gpt-4.1"
endpoint="https://mihai-azure-ai-search-basic-tier.search.windows.net"

agent = KnowledgeAgent(
    name=agent_name,
    models=[
        KnowledgeAgentAzureOpenAIModel(
            azure_open_ai_parameters=AzureOpenAIVectorizerParameters(
                resource_url=resource_url,
                deployment_name=deployment_name,
                model_name=model_name
            )
        )
    ],
    target_indexes=[
        KnowledgeAgentTargetIndex(
            index_name=index_name,
            default_reranker_threshold=2.5
        )
    ],
)

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index_client.create_or_update_agent(agent)

## Query an agent in Azure AI Search

In [6]:
from azure.search.documents.agent import KnowledgeAgentRetrievalClient
from azure.search.documents.agent.models import KnowledgeAgentRetrievalRequest, KnowledgeAgentMessage, KnowledgeAgentMessageTextContent, KnowledgeAgentIndexParams
from azure.search.documents.indexes.models import KnowledgeAgent, KnowledgeAgentAzureOpenAIModel, KnowledgeAgentTargetIndex, KnowledgeAgentRequestLimits, AzureOpenAIVectorizerParameters
from azure.search.documents.indexes import SearchIndexClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

credential = DefaultAzureCredential()
index_name = "dev-mihai-index"
agent_name = "dev-mihai-index-agent"
resource_url="https://cog-dcnxgznrq3wmc.openai.azure.com/"
deployment_name="gpt-4.1"
model_name="gpt-4.1"
endpoint="https://mihai-azure-ai-search-basic-tier.search.windows.net"

instructions = """
A Q&A agent that can answer questions about the Earth at night.
Sources have a JSON format with a ref_id that must be cited in the answer.
If you do not have the answer, respond with "I don't know".
"""

messages = [
    {
        "role": "system",
        "content": instructions
    }
]

agent_client = KnowledgeAgentRetrievalClient(endpoint=endpoint, agent_name=agent_name, credential=credential)

messages.append({
    "role": "user",
    "content": """
    cine e bituleanu?
    """
})

retrieval_result = agent_client.retrieve(
    retrieval_request=KnowledgeAgentRetrievalRequest(
        messages=[KnowledgeAgentMessage(role=msg["role"], 
        content=[KnowledgeAgentMessageTextContent(text=msg["content"])]) for msg in messages if msg["role"] != "system"],
        target_index_params=[KnowledgeAgentIndexParams(index_name=index_name, include_reference_source_data="true", reranker_threshold=0)]
    )
)
messages.append({
    "role": "assistant",
    "content": retrieval_result.response[0].content[0].text
})

import textwrap

print("Response")
print(textwrap.fill(retrieval_result.response[0].content[0].text, width=120))


In [ ]:
# Convert the retrieval_result response text to string and split by ',"', then print each part on a separate row
response_text = retrieval_result.response[0].content[0].text
for part in response_text.split(',"'):
    print(part)

In [ ]:
print(retrieval_result)

In [14]:
import textwrap
import json

# print("Response")
# print(textwrap.fill(retrieval_result.response[0].content[0].text, width=12000000))

# print("Activity")
# print(json.dumps([a.as_dict() for a in retrieval_result.activity], indent=2))

print("Results")
print(json.dumps([r.as_dict() for r in retrieval_result.references], indent=2))

Results
[
  {
    "type": "AzureSearchDoc",
    "id": "0",
    "activity_source": 1,
    "doc_key": "a322355a86f1_aHR0cHM6Ly9taWhhaXJhZ3N0b3JhZ2VhY2NvdW50LmJsb2IuY29yZS53aW5kb3dzLm5ldC9kZXYtbWloYWktY29udGFpbmVyLzIxODczLnBkZg2_pages_1",
    "source_data": {
      "title": "21873.pdf",
      "chunk_id": "a322355a86f1_aHR0cHM6Ly9taWhhaXJhZ3N0b3JhZ2VhY2NvdW50LmJsb2IuY29yZS53aW5kb3dzLm5ldC9kZXYtbWloYWktY29udGFpbmVyLzIxODczLnBkZg2_pages_1",
      "chunk": "performantele\tde\tieri,\tsa\tdevii\t\ntot\tmai\tbun,\tspre\tpropria\tsatisfactie\tprofesionala,\tspre\tsatisfactia\t\ncolegilor\ttai,\ta\tactionarilor\tnostri,\tspre\tsatisfactia\tclientilor.\t\n\nInca\to\tdata\tte\tfelicit\tpentru\tca\tte-ai\talaturat\techipei\tnoastre\tsi\titi\tdoresc\tmult\tsucces!\n\nI. MESAJUL PENTRU TINE\n\n5\n\nEmilian Bituleanu,  \nDirector General Libra Internet Bank\n\n\n\nLibra\t Internet\t Bank\t a\t fost\t infiintata\t in\t anul\t 1996\t\nsi\t este\t persoana\t juridica\t romana\t supusa\t legilor\t\nromane\t